In [175]:
import math 

In [176]:
def cal_entropy(lists,total_instances):
    p_label = 0
    e_label = 0
    instances = len(lists)
    for list in lists:
        if list[0] == 'p':
            p_label += 1
        elif list[0] == 'e':
            e_label += 1
    x = p_label/instances
    y = e_label/instances
    if x != 0 and y != 0:
        output = -(x) * math.log(x,2) - (y * math.log(y,2))
    elif x == 0:
        output = -(y * math.log(y,2))
    else:
        output = -(x * math.log(x,2)) 
    report = (instances/float(total_instances)) * output
    return report

In [177]:
def cal_expected_entropy(lists,attribute_num):
    total_instances = len(lists)
    labels = {}
    expected_entropy = 0
    for line in lists:
        if line[attribute_num] not in labels:
            labels[line[attribute_num]] = [line]
        else:
            labels[line[attribute_num]].append(line)
    for value in labels.values():
        expected_entropy += cal_entropy(value,total_instances)
    return expected_entropy,labels

In [178]:
def pick_highest_IG(lists):
    current_entropy = cal_entropy(lists,len(lists))
    highest_IG = 0
    best_attribute = 99
    classifier_dict = {}
    for i in range(1,len(lists[0])):
        expected_entropy,labels = cal_expected_entropy(lists,i)
        IG = current_entropy - expected_entropy
        if IG > highest_IG:
            highest_IG = IG
            best_attribute = i
            classifier_dict = labels
    return best_attribute,classifier_dict

In [179]:
def create_decision_tree(dataset,depth,stopping_depth):
    output_list = [i[0] for i in dataset]
    count_dict = {'e': 0, 'p': 0}
    for i in dataset:
        count_dict[i[0]] += 1
    find_max = max(count_dict, key=count_dict.get)
    if depth == stopping_depth:
        return find_max
    if all(i == output_list[0] for i in output_list):
        return output_list[0]
    if len(dataset[0]) == 2:
        return find_max
    attribute_index,classifier_dict = pick_highest_IG(dataset)
    attribute_name = mushroom_labels[attribute_index]
    del(mushroom_labels[attribute_index])
    for i in classifier_dict.values():
        for j in i:
            del(j[attribute_index])
    decision_tree = {attribute_name:{}}
    for key,value in classifier_dict.items():
        decision_tree[attribute_name][key] = create_decision_tree(value,depth+1,stopping_depth)
    return decision_tree

In [180]:
def test(tree, test_dataset, labels):
    correct = 0
    for count in range(len(test_dataset)):
        attribute = list(tree.keys())[0]
        values_dict = tree[attribute]
        str_value = test_dataset[count][labels.index(attribute)]
        if str_value in values_dict:
            currentDict = values_dict[str_value]
        else:
            count_dict = {'e': 0, 'p': 0}
            for value in values_dict.values():
                if type(value) != dict:
                    count_dict[value] += 1
            currentDict = max(count_dict, key=count_dict.get)
        while type(currentDict) == dict:
            attribute = list(currentDict.keys())[0]
            values_dict = currentDict[attribute]
            str_value = test_dataset[count][labels.index(attribute)]
            if str_value in values_dict:
                currentDict = values_dict[str_value]
            else:
                count_dict = {'e': 0, 'p': 0}
                for value in values_dict.values():
                    if type(value) != dict:
                        count_dict[value] += 1
                currentDict = max(count_dict, key=count_dict.get)
        if test_dataset[count][0] == currentDict:
            correct += 1
    output = correct*100/len(test_dataset)
    return output

In [181]:
def hold_out(dataset,num):
    len_train = int(len(dataset)*num)
    train_set = mushroom[:len_train]
    test_set = mushroom[len_train:]
    return train_set,test_set

In [182]:
def visualize(root, indent=0):
    if type(root) == dict:
        for k, v in root.items():
            print(" "* indent + f"{k}:")
            visualize(v, indent+2)
    else:
        print(" "*indent + repr(root))

In [183]:
def calcu_train_test_error(train_set,test_set,tree,mushroom_labels):
    test_error = 100 - test(tree, test_set, mushroom_labels_copy)
    train_error = 100 - test(tree, train_set, mushroom_labels_copy)
    print("The test error percentage is :",test_error)
    print("The training error percentage is :",train_error)
    return

In [184]:
file = open('agaricus-lepiota.data')
mushroom = [line.strip().split(',') for line in file.readlines()]
mushroom_labels = ['label','cap-shape', 'cap-surface','cap-color', 'bruises?' , 'odor', 'gill-attachment', 'gill-spacing', 'gill-size',
          'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 
          'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
mushroom_labels_copy = mushroom_labels.copy()

trainset_precent = 0.7
train_set,test_set = hold_out(mushroom,trainset_precent)
tree = create_decision_tree(train_set,0,3)
visualize(tree)
print("The accurancy percentage is :",test(tree, test_set, mushroom_labels_copy))
calcu_train_test_error(train_set,test_set,tree,mushroom_labels_copy)

odor:
  p:
    'p'
  a:
    'e'
  l:
    'e'
  n:
    spore-print-color:
      n:
        'e'
      k:
        'e'
      w:
        habitat:
          w:
            'e'
          l:
            'e'
          d:
            'p'
      h:
        'e'
      r:
        'p'
  f:
    'p'
  c:
    'p'
  y:
    'p'
  s:
    'p'
The accurancy percentage is : 99.30270713699754
The test error percentage is : 0.6972928630024597
The training error percentage is : 63.278227224762574
